In [1]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

# file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'

# image = np.loadtxt(file)
# file = '/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/83-100/aligned_hdf5/1859 HAADF 33.7 nm.emd.hspy'
file = '/home/zzhang/OneDrive/code/qem/examples/1859 HAADF 33.7 nm.emd.hspy'
s = hs.load(file)
image = s.data
# image = np.load(file)
# dx= 0.1645429228960236	
dx = s.axes_manager[1].scale *10
# dx = 0.36254824166962796
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [50]:
model.plot()

/home/zzhang/OneDrive/code/qem/qem/image_fitting.py:520: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [2]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05)
sr_coords = model.coordinates
# model.select_region(invert_selection=False)

Updated peak locations.


In [30]:
model.find_peaks(min_distance=4,threshold_abs=11500, threshold_rel=0.001)
total_coords = model.coordinates

Updated peak locations.


In [66]:
# ti_coords is the difference between sr_coords and total_coords
ti_coords = np.array([coord for coord in total_coords if np.linalg.norm((coord - sr_coords),axis=1).min()>8])
# remove the coordinates that are too close between themselves in ti_coords
from qem.utils import remove_close_coordinates
ti_coords,_ = remove_close_coordinates(ti_coords,12)

In [62]:
len(ti_coords)

8898

In [67]:
combined_coords = np.concatenate((sr_coords,ti_coords))
combined_coords, keep_mask = remove_close_coordinates(combined_coords,8)
model.coordinates = combined_coords
model.atom_types = np.concatenate((np.zeros(len(sr_coords)).astype(int),np.ones(len(ti_coords)).astype(int)))[keep_mask]

In [68]:
model.plot_coordinates()

In [3]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20, reciprocal=True)

Origin selected: [ 590. 1527.]
Vector a selected: [23. -1.]
Vector b selected: [ -1. -26.]
In pixel: Origin: [ 590. 1527.], Vector a: [23. -1.], Vector b: [ -1. -26.]
In space: Origin: [ 97.08032451 251.25704326] A, Vector a: [ 3.78448723 -0.16454292] A, Vector b: [-0.16454292 -4.278116  ] A
Origin selected: [1024. 1024.]
Vector a selected: [84.  0.]
Vector b selected: [ 0. 84.]
In pixel: Origin: [1024. 1024.], Vector a: [84.  0.], Vector b: [ 0. 84.]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A
FFT real a: [24.38095238  0.        ] pixel, Real b: [ 0.         24.38095238] pixel


In [5]:
model.add_or_remove_peaks(min_distance=10)

Updated peak locations.


array([[  13.80952381,  176.80952381],
       [  14.        ,  201.61904762],
       [  20.19047619, 1878.19047619],
       ...,
       [2043.19047619,  469.19047619],
       [2043.19047619,  543.19047619],
       [2043.19047619,  592.19047619]])

In [5]:
model.plot_coordinates()

In [4]:
model.select_region()

INFO:root:Selected 12633 atoms out of 12780 atoms.


In [6]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=3)

Fitting random batch: 100%|██████████| 13/13 [01:15<00:00,  5.79s/it]
INFO:root:Convergence rate for pos_x = 0.5519554501488528
INFO:root:Convergence rate for pos_y = 0.5553501674103245
INFO:root:Convergence rate for height = 0.0010421374149760922
INFO:root:Convergence not reached
Fitting random batch: 100%|██████████| 13/13 [01:18<00:00,  6.00s/it]
INFO:root:Convergence rate for pos_x = 0.53887939453125
INFO:root:Convergence rate for pos_y = 0.5562744140625
INFO:root:Convergence rate for height = 0.001032892958816235
INFO:root:Convergence not reached
Fitting random batch: 100%|██████████| 13/13 [01:14<00:00,  5.71s/it]
INFO:root:Convergence rate for pos_x = 0.5547332763671875
INFO:root:Convergence rate for pos_y = 0.5438232421875
INFO:root:Convergence rate for height = 0.0009835923412637533
INFO:root:Convergence rate for sigma = 0.024197469943547018
INFO:root:Convergence not reached


{'pos_x': array([   8.77100372,    9.63935947,    9.26354599, ..., 2042.22595215,
        2041.33093262, 2041.95776367]),
 'pos_y': array([2011.58312988, 1718.59008789, 1742.63696289, ...,  396.64300537,
         519.61425781,  446.10925293]),
 'height': array([879.78626742, 838.40378904, 892.84402577, ..., 403.74998507,
        425.91893854, 406.90637232]),
 'sigma': array([4.36162262, 4.90197779]),
 'background': 11450.382337660049}

In [9]:
model.plot_fitting()

In [8]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [10]:
model.plot_scs()

In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')
model.find_peaks()
crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'])

Updated peak locations.


In [5]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.choose_lattice_vectors()
crystal_analyzer.read_cif(cif_file)

Origin selected: [ 891. 1321.]
Vector a selected: [22. -1.]
Vector b selected: [ -1. -26.]
In pixel: Origin: [ 891. 1321.], Vector a: [22. -1.], Vector b: [ -1. -26.]
In space: Origin: [146.6077443  217.36120115] A, Vector a: [ 3.6199443  -0.16454292] A, Vector b: [-0.16454292 -4.278116  ] A


Atoms(symbols='SrTi', pbc=True, cell=[3.94513, 3.94513, 3.94513], spacegroup_kinds=...)

In [6]:
crystal_analyzer.unitcell_mapping()

(array([[ 891.      , 1321.      ,    0.      ],
        [ 901.5     , 1307.5     ,    1.972565]]),
 array([0, 1]))